Chatbot com memoria e template com variaveis

In [ ]:
# Carregar a chave de API do Google
import os
from dotenv import load_dotenv,find_dotenv

load_dotenv(find_dotenv())
_ = os.getenv('GOOGLE_API_KEY')

In [ ]:
# biblioteca para carregar o modelo da LLM
from langchain.chat_models import init_chat_model

# biblioteca para enviar mensagens para o modelo como humano
from langchain_core.messages import HumanMessage

# bibliotecas para adicionar histórico de mensagens ao chatbot
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph


In [ ]:
# Define o Modelo da LLM
model = init_chat_model("gemini-2.0-flash-lite", model_provider="google_genai")

In [ ]:
# biblioteca para passar templates de contexto
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um assistente útil. Responda a todas as perguntas da melhor forma possível em {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [ ]:
# bibliotecas para criar uma classe de estado e enviar variáveis para o chatbot
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


In [ ]:

# Define a new graph
workflow = StateGraph(state_schema=State)

# Define the function that calls the model
def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}

# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "thread_id"}}

In [ ]:

query = "Iai macho, tudo suave?"
language = "Português Cearense"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages, "language": language}, config)

output["messages"][-1].content  # output contains all messages in state

In [ ]:
output